# Function Generation for the Training of λ-Nets

## Specitication of Experiment Settings

In [1]:
#######################################################################################################################################
###################################################### CONFIG FILE ####################################################################
#######################################################################################################################################
sleep_time = 0 #minutes


config = {
    'function_family': {
        'maximum_depth': 5,
        'beta': 1,
        'decision_sparsity': 1,
        'fully_grown': True,
        'dt_type': 'vanilla', #'vanilla', 'SDT'
    },
    'data': {
        'number_of_variables': 9, 
        'num_classes': 2,
        'categorical_indices': [0,1,2,3],
        
        'function_generation_type': 'random_decision_tree_trained', # 'make_classification', 'make_classification_trained', 'random_decision_tree', 'random_decision_tree_trained'
        'objective': 'classification', # 'regression'
        
        'x_max': 1,
        'x_min': 0,
        'x_distrib': 'uniform', #'normal', 'uniform',       
                
        'lambda_dataset_size': 5000, #number of samples per function
        'number_of_generated_datasets': 10000,
    }, 
    'computation':{
        'n_jobs': -3,
        'use_gpu': False,
        'gpu_numbers': '0',
        'RANDOM_SEED': 42,   
    }
}


In [2]:
#######################################################################################################################################
########################################### IMPORT GLOBAL VARIABLES FROM CONFIG #######################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['computation'])

## Imports

In [3]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

from itertools import product       # forms cartesian products
from more_itertools import random_product 
from tqdm import tqdm_notebook as tqdm
import pickle

import numpy as np

import pandas as pd
from joblib import Parallel, delayed

import random 
from random import sample 

import os
import sys

from sklearn.datasets import make_regression
from sklearn.preprocessing import MinMaxScaler

from sympy import Symbol, sympify

        
import seaborn as sns
        
import random 

import warnings

from time import perf_counter



from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from utilities.DecisionTree_BASIC import SDT
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
from matplotlib import pyplot as plt
from IPython.display import Image

import numpy as np
import torch
np.set_printoptions(suppress=True)
torch.set_printoptions(sci_mode=False)

In [4]:
#######################################################################################################################################
###################################################### SET VARIABLES + DESIGN #########################################################
#######################################################################################################################################
os.environ['CUDA_VISIBLE_DEVICES'] = ''

sns.set_style("darkgrid")

random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
    
    
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.max_columns', 500)

warnings.filterwarnings('ignore')

In [5]:
from utilities.utility_functions import *
#######################################################################################################################################
####################################################### CONFIG ADJUSTMENTS ############################################################
#######################################################################################################################################

#######################################################################################################################################
################################################## UPDATE VARIABLES ###################################################################
#######################################################################################################################################
globals().update(config['function_family'])
globals().update(config['data'])
globals().update(config['computation'])

#initialize_utility_functions_config_from_curent_notebook(config)

#######################################################################################################################################
###################################################### PATH + FOLDER CREATION #########################################################
#######################################################################################################################################
globals().update(generate_paths(config, path_type='data_creation'))
generate_directory_structure()

#######################################################################################################################################
############################################################ SLEEP TIMER ##############################################################
#######################################################################################################################################
sleep_minutes(sleep_time)

In [6]:
print(path_identifier_function_data)

lNetSize5000_numDatasets10000_var9_class2_random_decision_tree_trained_xMax1_xMin0_xDistuniform_cat0-1-2-3_depth5_beta1_decisionSpars1_vanilla_fullyGrown


# Function Generation

In [7]:
if function_generation_type == 'random_decision_tree':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_random_decision_tree)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))
elif function_generation_type == 'random_decision_tree_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_random_decision_tree_trained)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))  
elif function_generation_type == 'make_classification':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_make_classification)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets)) 
elif function_generation_type == 'make_classification_trained':
    parallel = Parallel(n_jobs=n_jobs, verbose=3, backend='loky')
    result_list = parallel(delayed(generate_data_make_classification_decision_tree_trained)(config, seed=RANDOM_SEED+i) for i in range(number_of_generated_datasets))    

function_identifier_list = generate_decision_tree_identifier(config)  
identifier_series_list = [pd.Series(result[0],  index=function_identifier_list) for result in result_list]

function_df = pd.DataFrame(data=np.array([result[0] for result in result_list]), columns=function_identifier_list)

X_data_list = [[identifier_series, pd.DataFrame(result[1], columns=['x' + str(i) for i in range(number_of_variables)])] for identifier_series, result in zip(identifier_series_list, result_list)]
y_data_list = [[identifier_series, pd.DataFrame(result[2], columns=['result'])] for identifier_series, result in zip(identifier_series_list, result_list)]
y_data_raw_list = [[identifier_series, pd.DataFrame(result[3], columns=['result_raw'])] for identifier_series, result in zip(identifier_series_list, result_list)]


[Parallel(n_jobs=-3)]: Using backend LokyBackend with 22 concurrent workers.
[Parallel(n_jobs=-3)]: Done  84 tasks      | elapsed:   29.1s
[Parallel(n_jobs=-3)]: Done 244 tasks      | elapsed:   32.5s
[Parallel(n_jobs=-3)]: Done 468 tasks      | elapsed:   35.2s
[Parallel(n_jobs=-3)]: Done 756 tasks      | elapsed:   37.8s
[Parallel(n_jobs=-3)]: Done 1108 tasks      | elapsed:   41.1s
[Parallel(n_jobs=-3)]: Done 1524 tasks      | elapsed:   44.7s
[Parallel(n_jobs=-3)]: Done 2004 tasks      | elapsed:   49.1s
[Parallel(n_jobs=-3)]: Done 2548 tasks      | elapsed:   53.7s
[Parallel(n_jobs=-3)]: Done 3156 tasks      | elapsed:   58.9s
[Parallel(n_jobs=-3)]: Done 3828 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-3)]: Done 4564 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-3)]: Done 5364 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-3)]: Done 6228 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-3)]: Done 7156 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-3)]: Done 8148 tasks      |

In [8]:
function_df.head()

,feat0,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,feat11,feat12,feat13,feat14,feat15,feat16,feat17,feat18,feat19,feat20,feat21,feat22,feat23,feat24,feat25,feat26,feat27,feat28,feat29,feat30,split0,split1,split2,split3,split4,split5,split6,split7,split8,split9,split10,split11,split12,split13,split14,split15,split16,split17,split18,split19,split20,split21,split22,split23,split24,split25,split26,split27,split28,split29,split30,lp0,lp1,lp2,lp3,lp4,lp5,lp6,lp7,lp8,lp9,lp10,lp11,lp12,lp13,lp14,lp15,lp16,lp17,lp18,lp19,lp20,lp21,lp22,lp23,lp24,lp25,lp26,lp27,lp28,lp29,lp30,lp31
0,0.005,0.571,0.011,0.238,0.918,0.862,0.276,0.129,-1.000,0.500,-1.000,0.226,-1.000,0.361,0.305,-1.000,-1.000,-1.000,-1.000,0.003,0.597,-1.000,-1.000,0.008,0.283,-1.000,-1.000,0.077,0.403,0.304,0.360,7.000,5.000,7.000,5.000,6.000,8.000,7.000,5.000,-1.000,0.000,-1.000,8.000,-1.000,6.000,7.000,-1.000,-1.000,-1.000,-1.000,7.000,5.000,-1.000,-1.000,7.000,4.000,-1.000,-1.000,8.000,6.000,7.000,7.000,1.000,1.000,0.000,0.000,1.000,1.000,1.000,1.000,0.667,0.000,0.000,1.000,0.000,0.000,0.000,0.000,1.000,0.200,0.500,0.091,1.000,1.000,1.000,1.000,0.792,0.568,0.279,0.518,0.409,0.000,0.572,0.488
1,0.799,0.277,0.986,0.930,0.677,0.994,0.771,0.037,0.394,0.031,0.703,0.014,0.263,-1.000,-1.000,0.838,0.039,0.933,0.723,0.500,0.783,0.297,0.121,0.005,0.015,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,6.000,6.000,7.000,8.000,4.000,5.000,4.000,7.000,5.000,7.000,4.000,8.000,8.000,-1.000,-1.000,5.000,4.000,4.000,5.000,1.000,6.000,6.000,8.000,8.000,4.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,0.750,0.400,0.244,0.477,0.394,1.000,0.108,0.360,0.529,0.206,0.544,0.370,0.833,0.206,0.309,0.515,0.400,1.000,0.875,0.564,1.000,1.000,0.000,0.000,1.000,1.000,1.000,1.000,0.000,0.000,0.000,0.000
2,0.005,0.500,0.012,0.744,-1.000,0.369,0.658,0.659,-1.000,-1.000,-1.000,0.002,0.021,0.050,0.066,0.747,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,0.584,-1.000,0.635,0.920,0.055,0.036,0.891,8.000,0.000,5.000,6.000,-1.000,8.000,4.000,6.000,-1.000,-1.000,-1.000,5.000,6.000,5.000,7.000,4.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,-1.000,7.000,-1.000,4.000,6.000,5.000,5.000,8.000,1.000,0.500,0.000,0.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.000,0.444,0.000,0.000,0.844,1.000,0.423,0.083,0.889,0.515,0.000,0.586,0.465,0.366
3,0.909,0.019,0.117,0.005,0.580,0.835,0.786,0.201,0.476,0.096,0.734,0.020,0.822,0.082,0.404,-1.000,0.468,0.016,0.401,0.868,0.008,0.740,0.760,0.018,0.786,0.948,-1.000,0.683,0.741,0.414,0.500,5.000,4.000,7.000,4.000,6.000,8.000,6.000,5.000,6.000,6.000,6.000,7.000,4.000,4.000,4.000,-1.000,8.000,4.000,5.000,8.000,8.000,7.000,6.000,7.000,8.000,8.000,-1.000,7.000,6.000,8.000,0.000,0.000,0.000,1.000,0.222,0.775,1.000,0.750,0.333,0.509,0.723,0.857,0.455,0.586,0.464,0.354,0.504,0.143,1.000,0.000,0.333,1.000,0.250,0.000,0.000,0.455,0.917,0.419,0.682,0.267,0.050,0.227,0.571
4,0.065,0.609,0.014,0.570,0.684,0.207,0.078,0.809,0.892,0.854,0.005,0.010,0.177,0.352,0.082,0.219,0.554,-1.000,-1.000,-1.000,-1.000,0.553,0.694,0.937,-1.000,-1.000,0.546,0.929,0.111,0.865,0.284,7.000,5.000,8.000,5.000,5.000,6.000,7.000,6.000,6.000,6.000,7.000,8.000,4.000,4.000,7.000,5.000,5.000,-1.000,-1.000,-1.000,-1.000,6.000,5.000,7.000,-1.000,-1.000,4.000,8.000,6.000,5.000,7.000,0.639,0.434,0.821,0.000,0.000,0.000,1.000,1.000,1.000,1.000,0.000,0.000,0.000,0.600,0.000,0.716,0.000,0.500,1.000,1.000,1.000,1.000,0.481,0.833,0.000,1.000,0.000,0.447,0.923,0.000,0.458,0.501


In [9]:
X_data_list[0][0].head(10)

feat0    0.005
feat1    0.571
feat2    0.011
feat3    0.238
feat4    0.918
feat5    0.862
feat6    0.276
feat7    0.129
feat8   -1.000
feat9    0.500
dtype: float64

In [10]:
X_data_list[0][1].head(10)

,x0,x1,x2,x3,x4,x5,x6,x7,x8
0,0.000,1.000,1.000,1.000,0.156,0.156,0.058,0.866,0.601
1,1.000,0.000,1.000,1.000,0.212,0.182,0.183,0.304,0.525
2,0.000,0.000,1.000,0.000,0.292,0.366,0.456,0.785,0.200
3,1.000,1.000,0.000,1.000,0.171,0.065,0.949,0.966,0.808
4,0.000,0.000,1.000,0.000,0.122,0.495,0.034,0.909,0.259
5,1.000,0.000,1.000,1.000,0.185,0.970,0.775,0.939,0.895
6,1.000,1.000,0.000,0.000,0.045,0.325,0.389,0.271,0.829
7,0.000,0.000,1.000,0.000,0.802,0.075,0.987,0.772,0.199
8,0.000,1.000,1.000,1.000,0.771,0.074,0.358,0.116,0.863
9,1.000,0.000,0.000,0.000,0.325,0.730,0.638,0.887,0.472


In [11]:
y_data_list[0][0].head(10)

feat0    0.005
feat1    0.571
feat2    0.011
feat3    0.238
feat4    0.918
feat5    0.862
feat6    0.276
feat7    0.129
feat8   -1.000
feat9    0.500
dtype: float64

In [12]:
y_data_list[1][1].head(10)

,result
0,0
1,0
2,1
3,0
4,0
5,0
6,0
7,0
8,1
9,0


In [13]:
y_data_list[2][1].head(10)

,result
0,0
1,0
2,0
3,0
4,1
5,0
6,0
7,0
8,1
9,0


In [14]:
y_data_raw_list[2][1].head(10)

,result_raw
0,0
1,0
2,0
3,0
4,1
5,0
6,0
7,0
8,1
9,0


In [15]:
plot_tree_from_parameters(y_data_list[0][0].values, config)

In [16]:
path_functions = './data/saved_function_lists/functions_' + path_identifier_function_data + '.csv'
function_df.to_csv(path_functions, index=False)

path_X_data = './data/saved_function_lists/X_data_' + path_identifier_function_data + '.pkl'
with open(path_X_data, 'wb') as f:
    pickle.dump(X_data_list, f)
    
path_y_data = './data/saved_function_lists/y_data_' + path_identifier_function_data + '.pkl'
with open(path_y_data, 'wb') as f:
    pickle.dump(y_data_list, f)
